# 🔷 CNN Baseline - Оптимізоване навчання

**Модель:** Simple Convolutional Neural Network  
**Параметри:** ~60,000  
**Характеристики:** Швидка, проста, мало параметрів  

---

## ⚖️ Стандартизовані параметри (однакові для всіх моделей):
- **Epochs:** 40 ✅
- **Batch size:** 64 ✅
- **Learning rate:** 0.001 ✅
- **Weight decay:** 1e-5 ✅
- **Optimizer:** Adam ✅
- **Scheduler:** ReduceLROnPlateau ✅
- **Dataset:** 1M+ puzzles (весь датасет) ✅

## 🔧 Архітектурні параметри (специфічні для CNN Baseline):
- **Hidden channels:** 64
- **Conv layers:** 5

**Очікуваний результат:** 85-88% cell accuracy, 30-35% board accuracy

## 📦 Крок 1: Налаштування середовища

In [ ]:
# Підключення Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

print("✅ Google Drive mounted!")

In [ ]:
# Встановлення залежностей
print("📦 Installing dependencies...")

# PyTorch (стабільна версія)
!pip uninstall torch torchvision torchaudio -y -q
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121 -q

# Додаткові бібліотеки
!pip install tqdm pandas numpy matplotlib scikit-learn -q

print("✅ Dependencies installed!")

In [ ]:
# Перевірка GPU
import torch

if torch.cuda.is_available():
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ GPU not available, will use CPU")

## 📂 Крок 2: Підготовка проєкту

In [ ]:
# Шлях до проєкту (змініть на свій!)
PROJECT_PATH = '/content/drive/MyDrive/Diplom/backend'

# Перехід у папку проєкту
os.chdir(PROJECT_PATH)
print(f"📂 Working directory: {os.getcwd()}")

# Перевірка наявності файлів
required_files = ['train.py', 'dataset.py', 'models/__init__.py', 'data/sudoku.csv']
for file in required_files:
    if os.path.exists(file):
        print(f"✅ {file}")
    else:
        print(f"❌ {file} - NOT FOUND!")

## 🎓 Крок 3: Навчання CNN Baseline

### Оптимальні параметри для максимального результату

In [ ]:
# === НАВЧАННЯ CNN BASELINE ===
# ⚖️ СТАНДАРТИЗОВАНІ ПАРАМЕТРИ (однакові для всіх моделей)

!python train.py \
    --model baseline \
    --epochs 40 \
    --batch-size 64 \
    --lr 0.001 \
    --hidden-channels 64 \
    --device cuda \
    --weight-decay 1e-5 \
    --limit 1000000 \
    --save-dir weights

# 🎯 ФІКСОВАНІ ПАРАМЕТРИ (для чесного порівняння):
# --epochs 40              : ОДНАКОВЕ для всіх моделей
# --batch-size 64          : ОДНАКОВЕ для всіх моделей
# --lr 0.001               : ОДНАКОВЕ для всіх моделей
# --weight-decay 1e-5      : ОДНАКОВЕ для всіх моделей
# --limit 1000000          : РІВНО 1М судоку (800К train + 200К val)
# 
# 🔧 Архітектурні параметри (специфічні для моделі):
# --hidden-channels 64     : визначено архітектурою

## 📊 Крок 4: Аналіз результатів

In [ ]:
import json
import matplotlib.pyplot as plt

# Завантаження історії навчання
with open('weights/baseline_history.json', 'r') as f:
    history = json.load(f)

# Підготовка даних
epochs = [h['epoch'] for h in history]
train_loss = [h['train']['loss'] for h in history]
val_loss = [h['val']['loss'] for h in history]
train_acc = [h['train']['cell_accuracy'] for h in history]
val_acc = [h['val']['cell_accuracy'] for h in history]
board_acc = [h['val']['board_accuracy'] for h in history]

# Візуалізація
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss
axes[0].plot(epochs, train_loss, label='Train Loss', marker='o')
axes[0].plot(epochs, val_loss, label='Val Loss', marker='s')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('CNN Baseline - Loss')
axes[0].legend()
axes[0].grid(True)

# Cell Accuracy
axes[1].plot(epochs, train_acc, label='Train Acc', marker='o')
axes[1].plot(epochs, val_acc, label='Val Acc', marker='s')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Cell Accuracy')
axes[1].set_title('CNN Baseline - Cell Accuracy')
axes[1].legend()
axes[1].grid(True)

# Board Accuracy
axes[2].plot(epochs, board_acc, label='Board Acc', marker='D', color='green')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Board Accuracy')
axes[2].set_title('CNN Baseline - Board Accuracy')
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.savefig('weights/baseline_training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

# Фінальні результати
print("\n" + "="*60)
print("📊 ФІНАЛЬНІ РЕЗУЛЬТАТИ CNN BASELINE")
print("="*60)
print(f"Best Validation Loss:       {min(val_loss):.4f}")
print(f"Best Cell Accuracy:         {max(val_acc):.2%}")
print(f"Best Board Accuracy:        {max(board_acc):.2%}")
print(f"Final Cell Accuracy:        {val_acc[-1]:.2%}")
print(f"Final Board Accuracy:       {board_acc[-1]:.2%}")
print("="*60)

## 🧪 Крок 5: Тестування моделі

In [ ]:
import torch
import numpy as np
from models import CNNBaseline

# Завантаження моделі
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CNNBaseline(hidden_channels=64).to(device)

checkpoint = torch.load('weights/baseline_best.pth', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"✅ Model loaded from epoch {checkpoint['epoch']}")
print(f"   Validation accuracy: {checkpoint['val_accuracy']:.2%}")

# Тестовий пазл
test_puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

# Inference
input_tensor = torch.tensor(test_puzzle, dtype=torch.long).unsqueeze(0).to(device)
with torch.no_grad():
    output = model(input_tensor)
    predictions = torch.argmax(output, dim=-1).squeeze().cpu().numpy() + 1
    
    # Confidence
    probs = torch.softmax(output, dim=-1)
    confidence = probs.max(dim=-1)[0].mean().item()

# Виведення результату
print("\n🎯 ТЕСТ ІНФЕРЕНСУ")
print(f"Confidence: {confidence:.2%}\n")
print("Solved puzzle:")
for row in predictions:
    print(' '.join(map(str, row)))

## 💾 Крок 6: Збереження результатів

In [ ]:
# Створення звіту
report = f"""
╔══════════════════════════════════════════════════════════╗
║         CNN BASELINE - ЗВІТ ПРО НАВЧАННЯ                ║
╚══════════════════════════════════════════════════════════╝

📊 АРХІТЕКТУРА
├─ Тип:               Simple CNN
├─ Параметри:         ~60,000
├─ Шари:              5 convolutional layers
└─ Hidden channels:   64

🎓 НАВЧАННЯ
├─ Epochs:            30
├─ Batch size:        128
├─ Learning rate:     0.001
├─ Optimizer:         Adam
├─ Scheduler:         ReduceLROnPlateau
└─ Device:            {device}

📈 РЕЗУЛЬТАТИ
├─ Best Val Loss:         {min(val_loss):.4f}
├─ Best Cell Accuracy:    {max(val_acc):.2%}
├─ Best Board Accuracy:   {max(board_acc):.2%}
├─ Final Cell Accuracy:   {val_acc[-1]:.2%}
└─ Final Board Accuracy:  {board_acc[-1]:.2%}

💡 ВИСНОВКИ
└─ CNN Baseline показує {max(val_acc):.1%} точності на клітинках.
   Це хороша базова лінія для порівняння з іншими моделями.
   Модель швидка і має мало параметрів.

📁 ФАЙЛИ
├─ weights/baseline_best.pth          - найкраща модель
├─ weights/baseline_last.pth          - остання модель
├─ weights/baseline_history.json      - історія навчання
└─ weights/baseline_training_curves.png - графіки

Дата: {history[-1].get('timestamp', 'N/A')}
"""

print(report)

# Збереження звіту
with open('weights/baseline_report.txt', 'w', encoding='utf-8') as f:
    f.write(report)

print("\n✅ Звіт збережено у weights/baseline_report.txt")

## ✅ Готово!

CNN Baseline навчена з оптимальними параметрами.  
Результати збережені у папці `weights/`.

**Наступний крок:** Запустіть інші ноутбуки для порівняння:
- `Train_CNN_Advanced.ipynb`
- `Train_GNN.ipynb`
- `Train_RNN.ipynb`